In [1]:
import numpy as np
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import math
import pywt
import scipy
from scipy import ndimage

# Create some random data to be tested for WaveCluster

In [2]:
#Generate random points
#centers = [[1, 1], [-1, -1], [1, -1]]
X, y= make_blobs(n_samples=1000, n_features = 3, cluster_std=0.4, random_state=0)

X = StandardScaler().fit_transform(X)

In [10]:
X.shape

(1000, 3)

In [4]:
#plt.scatter(X[:,0],X[:,1])

# Segment the space into cubes and bin the data

In [4]:
#Compute and/or set some variables
levels = 1   #set levels of DWT.  May end up being less
n_bins = 2**8
d = X.shape[1] #number of dimensions

In [5]:
H = np.histogramdd(X, bins=n_bins)
data_quant = H[0]

In [7]:
data_quant.shape

(256, 256, 256)

In [9]:
data_quant

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [8]:
import sys
sys.getsizeof(data_quant)

128

# Compute the DWT

In [7]:
#pywt.wavelist()
#Information : http://wavelets.pybytes.com/

In [11]:
#Select a wavelet.  Probably want Haar for average values
wave = 'haar'
print(pywt.Wavelet(wave))

Wavelet haar
  Family name:    Haar
  Short name:     haar
  Filters length: 2
  Orthogonal:     True
  Biorthogonal:   True
  Symmetry:       asymmetric
  DWT:            True
  CWT:            False


In [12]:
max_level = pywt.dwt_max_level(data_len = n_bins , filter_len = pywt.Wavelet(wave).dec_len) #figure out max levels to loop over

In [13]:
#Perform dwt on quantized data.
wp = pywt.wavedecn(data=data_quant, wavelet=wave, level = min(levels,max_level))
#wp = pywt.dwtn(data=data_quant, wavelet=wave)

In [14]:
#I cannot see a way around computing DWT twice in the thresholding step
wp_c = pywt.wavedecn(data=data_quant, wavelet=wave, level = min(levels,max_level))

In [15]:
wp[0].size

2097152

# Threshold the results of DWT

In [28]:
#Pick a threshold value.  Compute some details for loops
# For climate data, we probably only want the approximate (low frequency) coefficents to cluster. 
epsilon = .0001
keys = wp[1].keys()  #build keys to loop over

In [29]:
#Threshold the DWT
wp_c[0][abs(wp_c[0])<epsilon] = 0
wp_c[0][abs(wp_c[0])>=epsilon] = 1

#for i in range(1,min(levels,max_level)+1):
#    for k in keys:
#        wp_c[i][k][abs(wp_c[i][k])<epsilon] = 0
#        wp_c[i][k][abs(wp_c[i][k])>=epsilon] = 1

# Find connected components

In [25]:
#Compute the connected components of each thresholed DWT.  Adjacnecy is determined by ``four'' connectivity.
component = ndimage.label(wp_c[0])[0]

#wp_c[0] = component[0]
#for i in range(1,min(levels,max_level)+1):
#    for k in keys:
#        component = ndimage.label(wp_c[i][k])
#        wp_c[i][k] = component[0]

# Create Lookup Table

In [ ]:
#The following lookup table is only built to handle the Haar wavelet.
#First, label data_quant classes
classes = np.repeat(component, levels + 1)

if data_quant.size % 2 ==1:  #If odd number of elements in array, need to deal with right endpoint
    classes = classes[:-1]

In [ ]:
classes

In [ ]:
#Invert the histogram
np.digitize(X, H[1][0])

# Trash

In [ ]:
#Create the bins
#bin_list = []
#for i in list(range(d)):
#    dim_min = math.floor(min(X[:,i]))
#    dim_max = math.ceil(max(X[:,i]))
#    dim_bin = np.linspace(start=dim_min, stop=dim_max, num=n_bins)
#    bin_list.append(dim_bin)
#    
#bin_list = np.stack(bin_list, axis=0)

In [ ]:
#Quantize data into the bins
#data_quant = []
#index_key = []
#for b in list(range(d)):
#    inds = np.digitize(X[:,b], bin_list[b])
#    index_key.append(inds)
#    data_quant.append(np.bincount(inds, minlength = n_bins))
#
#index_key = np.stack(index_key,axis=0)
#data_quant = np.stack(data_quant, axis=0)